In [4]:
from osgeo import gdal, ogr, osr
import geopandas as gpd
import numpy as np                  

In [5]:
def vectorize_predictions(raster_path, dest_path, epsg=5253):
    image = gdal.Open(raster_path)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(epsg)
    driver = ogr.GetDriverByName('ESRI Shapefile')
    vector_src = driver.CreateDataSource(dest_path)
    layer = vector_src.CreateLayer('polygon', geom_type=ogr.wkbPolygon, srs=srs)
    layer.CreateField(ogr.FieldDefn('id', ogr.OFTInteger))
    img_band = image.GetRasterBand(1)
    img_band.SetNoDataValue(0)
    gdal.Polygonize(img_band, img_band, layer,
                    0, [], callback=None)
    vector_src.Destroy()

In [6]:

def simplify_vector(shp_path, tolerance_list, dst_base_path, min_threshold=0.1, apply_threshold=True):
    shp_df = gpd.read_file(shp_path)
    file_ids = open(dst_base_path +"\\"+ "simplification_results.txt", "w")        
    for k in tolerance_list:
        new_df = shp_df.geometry.simplify(tolerance=k, preserve_topology=True)
        diff = new_df.area - shp_df.area
        diff = np.mean(diff)
        if apply_threshold and abs(diff) <= min_threshold:
            dst_path = dst_base_path + "\\" + f"simplified_tolerance_{k}_area_diff_{diff}.shp"
            new_df.to_file(dst_path)
            file_ids.write("Tolerance: " + str(k) + "\t" + "Mean Diff: " + str(diff) + "\n")
        if not apply_threshold:
            dst_path = dst_base_path + "\\" +  f"simplified_tolerance_{k}_area_diff_{diff}.shp"
            new_df.to_file(dst_path)
            file_ids.write("Tolerance: " + str(k) + "\t" + "Mean Diff: " + str(diff) + "\n")
    file_ids.close()

In [7]:
file = "Unet_seresnet18_100epoch_16batch_nDSM"
path = f"results_trueop\\{file}\\{file}.tif"
shp_path = f"results_trueop\\test\\{file}.shp"
vectorize_predictions(path, shp_path, epsg=5253)

In [10]:
tolerance_list = np.arange(2.0, 0.1, -0.05)
shp_path = f"results_trueop\\test\\{file}.shp"
dst_base_path = "results_trueop\\test"
simplify_vector(shp_path, tolerance_list, dst_base_path, min_threshold=0.1, apply_threshold=False)